In [186]:
import psycopg
import json
import folium
import numpy as np
import numpy.ma as ma
import pandas as pd

In [189]:
conn = psycopg.connect(
        host="localhost",
        dbname="postgis",
        user="username",
        password="password",
    )

In [190]:
route = '4'

cur = conn.execute("""
    SELECT DISTINCT ON (trips.shape_id) trips.shape_id,trips.trip_id, trips.direction_id, trips.trip_headsign FROM trips
    JOIN routes ON trips.route_id=routes.route_id
    WHERE routes.route_short_name=%s
    """, (route,)
)
shapes = cur.fetchall()
shapes

[('40066', '64325884', 0, 'HUNTINGTON'),
 ('40072', '64341839', 0, 'HUNTINGTON')]

In [191]:
route_index = 0
shape = shapes[route_index][0]

cur = conn.execute("""
    SELECT ST_AsGeoJSON(
        shapes_aggregated.shape)
    FROM shapes_aggregated
    WHERE shapes_aggregated.shape_id=%s
    """, (shape,)
)


# cur = conn.execute("""
#  SELECT ST_AsGeoJSON(
#          ST_Transform(
#              ST_OffsetCurve(
#                  ST_Transform(
#                      ST_SetSRID(shapes_aggregated.shape,4326),
#                      3401
#                 ),10),
#           4326)
#          )
#     FROM shapes_aggregated
#     WHERE shapes_aggregated.shape_id=%s
#     """, (shape,)
# )


line = cur.fetchall()
geojson = json.loads(line[0][0])


routeCoord = [v[::-1] for v in geojson["coordinates"]]
m = folium.Map(location=[51.05084309996687, -114.07251113843348], zoom_start=12)

folium.PolyLine(routeCoord, color="red", weight=5).add_to(m)
m

In [178]:
trip_id = shapes[route_index][1]
cur = conn.execute(
    """
    SELECT json_build_object(
        'type', 'FeatureCollection',
        'features', json_agg(ST_AsGeoJSON(t.*)::json)) FROM
    (SELECT stops.stop_loc, b.stop_sequence, b.shape_dist_traveled
    FROM (SELECT stop_times.stop_id, stop_times.stop_sequence, stop_times.shape_dist_traveled FROM stop_times
    WHERE stop_times.trip_id=%s
    ORDER BY stop_times.stop_sequence) as b
    JOIN stops ON b.stop_id=stops.stop_id) as t
    """,
    (trip_id,))

stops = cur.fetchall()[0][0]

In [179]:
coordinates = [i["geometry"]["coordinates"] for i in stops["features"]]
sequence = [i["properties"]["stop_sequence"] for i in stops["features"]]
dist_trav = [i["properties"]["shape_dist_traveled"] for i in stops["features"]]
for i, coord in enumerate(coordinates):
    folium.Marker(
        [coord[1], coord[0]],popup=f"Stop Sequence: {sequence[i]}, Dist Travelled: {dist_trav[i]}"
    ).add_to(m)
m

In [7]:
def check_increasing(distances):
    myseries = pd.Series(distances)
    return myseries.is_unique and myseries.is_monotonic_increasing

In [8]:
a = [0,2,5,4,2,7,100]

def lcs(numbers):
    n = len(numbers)
    dp = [1] * n
    for i in range(1, n):
        for j in range(i):
            if numbers[i] > numbers[j]:
                dp[i] = max(dp[i], dp[j] + 1)

    return dp

In [12]:
from itertools import pairwise

def fix_distances(stops, line_length):
    seq = lcs(stops)
    mask = np.zeros(len(seq))
    c = list(pairwise(seq))
    print(seq)
    for idx, i in enumerate(c):
        if i[1]>i[0]:
            mask[idx+1]=0
        elif i[1]==i[0]:
            mask[idx]=1
        else:
            mask[idx+1]=1

    mx = ma.masked_array(np.array(stops), mask)
    d = line_length-ma.masked_array(np.array(stops), np.logical_not(mask))
    return mx.filled(0)+d.filled(0)
    
b = fix_distances(distances, length)
print(length)
for i, dis in enumerate(distances):
    print(i+1, dis, b[i])

[1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 25, 25, 25, 26, 27, 28, 29, 30, 31, 32, 33, 26, 34, 25, 35, 25, 36, 37, 38, 39, 40, 41]
22510.393538010485
1 22510.375336322686 0.01820168779886444
2 606.4766629498312 606.4766629498312
3 841.4743492579211 841.4743492579211
4 5597.830336314389 5597.830336314389
5 5907.53569582972 5907.53569582972
6 6081.775564439091 6081.775564439091
7 6497.207180691421 6497.207180691421
8 6730.965805756818 6730.965805756818
9 6931.949544325634 6931.949544325634
10 7100.023607438465 7100.023607438465
11 7304.408964924137 7304.408964924137
12 7707.716084100702 7707.716084100702
13 7950.361767236309 7950.361767236309
14 8248.812327226424 8248.812327226424
15 8653.023909402109 8653.023909402109
16 8963.266245882854 8963.266245882854
17 9201.230257884454 9201.230257884454
18 9454.004356911631 9454.004356911631
19 9809.985832308894 9809.985832308894
20 10269.578838132895 10269.578838132895
21 10643.613021078565 10

In [10]:
# # Alberta projection = 3402
# # BC Projection = 3005

# cur = conn.execute("""    
#     ST_LineLocatePoint(
#         ST_SetSRID(shapes.shape,4326), 
#         shapes.stop_loc::geometry) * 
#         ST_Length(
#             ST_Transform(
#                 ST_SetSRID(shapesv.shape,4326),
#                 3402
#             )
#         ) as stops
#         FROM
#     (
#     SELECT stop_times.trip_id, trips.shape_id, stops.stop_loc, stop_times.stop_sequence, stop_times.stop_id, shapes_aggregated.shape FROM stop_times
#     JOIN stops ON stop_times.stop_id=stops.stop_id
#     JOIN trips ON stop_times.trip_id=trips.trip_id
#     JOIN shapes_aggregated ON trips.shape_id=shapes_aggregated.shape_id
#     WHERE stop_times.trip_id=%s
#     ORDER BY stop_times.stop_sequence) as shapes
#     """,
#     (shapes[route_index][1],))

# result = cur.fetchall()
# result

In [11]:
# Alberta projection = 3402
# BC Projection = 3005

cur = conn.execute("""
    
    SELECT ST_Length(
            ST_Transform(
                ST_SetSRID(shape_return.shape,4326),
                3402
            )
        ) as length,
    ST_LineLocatePoint(
        ST_SetSRID(shape_return.shape,4326), 
        stop_return.stop_loc::geometry) * 
        ST_Length(
            ST_Transform(
                ST_SetSRID(shape_return.shape,4326),
                3402
            )
        ) as stops
        FROM
    (
    SELECT stop_times.trip_id, trips.shape_id, stops.stop_loc, stop_times.stop_sequence, stop_times.stop_id FROM stop_times
    JOIN stops ON stop_times.stop_id=stops.stop_id
    JOIN trips ON stop_times.trip_id=trips.trip_id
    WHERE stop_times.trip_id=%s
    ORDER BY stop_times.stop_sequence
    ) AS stop_return
    JOIN (
    SELECT shapes_aggregated.shape, shapes_aggregated.shape_id
    FROM shapes_aggregated
    WHERE shapes_aggregated.shape_id=%s
    ) AS shape_return 
    ON stop_return.shape_id=shape_return.shape_id
    """,
    (shapes[route_index][1],shapes[route_index][0]))

result = cur.fetchall()
length = result[0][0]
distances = [i[1] for i in result]


print(check_increasing(distances))
print(length , distances)

stops = fix_distances(distances, length)
print(stops)
print(check_increasing(stops))

False
22510.393538010485 [22510.375336322686, 606.4766629498312, 841.4743492579211, 5597.830336314389, 5907.53569582972, 6081.775564439091, 6497.207180691421, 6730.965805756818, 6931.949544325634, 7100.023607438465, 7304.408964924137, 7707.716084100702, 7950.361767236309, 8248.812327226424, 8653.023909402109, 8963.266245882854, 9201.230257884454, 9454.004356911631, 9809.985832308894, 10269.578838132895, 10643.613021078565, 10901.340197296267, 11582.893747364773, 11714.633426178123, 12156.129675361317, 15985.885216472732, 15700.070284486672, 15476.387852462032, 12834.856003228018, 13210.818308260137, 13565.649103524252, 13902.422854710749, 14200.024627240118, 14436.322869017995, 14666.365440559239, 14786.97041952386, 14942.368320008269, 13103.081690425986, 15390.448864917635, 12712.32552783889, 15753.102449082777, 12249.45243154593, 16176.390081400505, 16644.076940444254, 16749.15356372122, 22024.632330407232, 22300.00315372718, 22510.375336322686]
[1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,

In [9]:
stop_distances = np.copy(distances)
end = stop_distances[-1]
for i in range(1,len(distances)-1):
    if (stop_distances[i-1]>stop_distances[i]):
        if(i<len(distances)/2):
            stop_distances[i-1]=end-stop_distances[i-1]
        else:
            stop_distances[i]=end-stop_distances[i]
        
print(check_increasing(stop_distances))


True


In [10]:
d = {'original': distances, 'modified': stop_distances, }
df = pd.DataFrame(d)
df

,original,modified
0,0.000000,0.000000
1,352.708020,352.708020
2,521.939473,521.939473
3,709.079693,709.079693
4,877.508449,877.508449
5,1077.579787,1077.579787
6,1671.672944,1671.672944
7,2138.069452,2138.069452
8,2636.701660,2636.701660
9,2984.635561,2984.635561


In [11]:
print(check_increasing(stop_distances))

True


In [163]:
# cur = conn.execute("""
#     WITH
#         shape_return AS (
#         SELECT 
#             shapes_aggregated.shape
#         FROM shapes_aggregated
#         WHERE shapes_aggregated.shape_id=%s
#         ),
#         stops_return AS (
#         SELECT 
#             stops.stop_loc
#         FROM (SELECT stop_times.stop_id, stop_times.stop_sequence FROM stop_times
#         WHERE stop_times.trip_id=%s
#         ORDER BY stop_times.stop_sequence) as b
#         JOIN stops ON b.stop_id=stops.stop_id limit 1
#         )
#     SELECT ST_LineLocatePoint(
#         (SELECT ST_SetSRID(shape_return.shape,4326) FROM shape_return), 
#         (SELECT stops_return.stop_loc::geometry FROM stops_return)
#         );
#     """,
#     (shapes[0][0],shapes[0][1]))

In [164]:
# cur = conn.execute("""
#     SELECT ST_LineLocatePoint(
#         ST_SetSRID(shape_return.shape,4326), 
#         stop_return.stop_loc::geometry) 
#         FROM
#     (
#     SELECT stop_times.trip_id, trips.shape_id, stops.stop_loc, stop_times.stop_sequence, stop_times.stop_id FROM stop_times
#     JOIN stops ON stop_times.stop_id=stops.stop_id
#     JOIN trips ON stop_times.trip_id=trips.trip_id
#     WHERE stop_times.trip_id=%s
#     ORDER BY stop_times.stop_sequence
#     ) AS stop_return
#     JOIN (
#     SELECT shapes_aggregated.shape, shapes_aggregated.shape_id
#     FROM shapes_aggregated
#     WHERE shapes_aggregated.shape_id=%s
#     ) AS shape_return 
#     ON stop_return.shape_id=shape_return.shape_id
#     """,
#     (shapes[0][1],shapes[0][0]))

# distances = cur.fetchall()
# distances = np.array([i[0] for i in distances])

# check_increasing(distances)
# distances

In [ ]:
# cur = conn.execute(
#     "SELECT ST_AsGeoJSON(shapes_aggregated.shape) "
#     "FROM (SELECT DISTINCT ON (trips.shape_id) trips.shape_id FROM trips " 
#     "JOIN routes ON trips.route_id=routes.route_id "
#     "WHERE routes.route_short_name=%s) AS b "
#     "JOIN shapes_aggregated ON b.shape_id = shapes_aggregated.shape_id ", (route,)
    
# )
